In [1]:
import pandas as pd

In [2]:
dataframe = pd.read_csv('readings.csv')

In [3]:
dataframe.head()

,date,cloud_cover,sunshine,global_radiation,max_temp,mean_temp,min_temp,precipitation,pressure,snow_depth
0,19790101,2.0,7.0,52.0,2.3,-4.1,-7.5,0.4,101900.0,9.0
1,19790102,6.0,1.7,27.0,1.6,-2.6,-7.5,0.0,102530.0,8.0
2,19790103,5.0,0.0,13.0,1.3,-2.8,-7.2,0.0,102050.0,4.0
3,19790104,8.0,0.0,13.0,-0.3,-2.6,-6.5,0.0,100840.0,2.0
4,19790105,6.0,2.0,29.0,5.6,-0.8,-1.4,0.0,102250.0,1.0


## Feature engineering
timestamp --> year, month, date

In [4]:
dataframe['date'] = dataframe['date'].astype(str)

dataframe['year'] = dataframe['date'].str[:4]
dataframe['month'] = dataframe['date'].str[4:6]
dataframe['day'] = dataframe['date'].str[6:8]

dataframe.drop(labels='date', axis=1, inplace=True)

In [5]:
dataframe

,cloud_cover,sunshine,global_radiation,max_temp,mean_temp,min_temp,precipitation,pressure,snow_depth,year,month,day
0,2.0,7.0,52.0,2.3,-4.1,-7.5,0.4,101900.0,9.0,1979,01,01
1,6.0,1.7,27.0,1.6,-2.6,-7.5,0.0,102530.0,8.0,1979,01,02
2,5.0,0.0,13.0,1.3,-2.8,-7.2,0.0,102050.0,4.0,1979,01,03
3,8.0,0.0,13.0,-0.3,-2.6,-6.5,0.0,100840.0,2.0,1979,01,04
4,6.0,2.0,29.0,5.6,-0.8,-1.4,0.0,102250.0,1.0,1979,01,05
...,...,...,...,...,...,...,...,...,...,...,...,...
15336,1.0,0.9,32.0,7.5,7.5,7.6,2.0,98000.0,NaN,2020,12,27
15337,7.0,3.7,38.0,3.6,1.1,-1.3,0.2,97370.0,NaN,2020,12,28
15338,7.0,0.0,21.0,4.1,2.6,1.1,0.0,98830.0,NaN,2020,12,29
15339,6.0,0.4,22.0,5.6,2.7,-0.1,0.0,100200.0,NaN,2020,12,30


## Data cleaning

Which columns contain NaN values?

In [11]:
def get_monthly_mean(df):
    # Compute group-wise means while preserving the original DataFrame shape
    grouped_mean = df.groupby(['year', 'month']).transform('mean')

    # Fill NaN values in the original DataFrame with these means
    df = df.fillna(grouped_mean)
    print(df)
    return df

In [14]:
dataframe.fillna(value=0)

,cloud_cover,sunshine,global_radiation,max_temp,mean_temp,min_temp,precipitation,pressure,snow_depth,year,month,day
0,2.0,7.0,52.0,2.3,-4.1,-7.5,0.4,101900.0,9.0,1979,01,01
1,6.0,1.7,27.0,1.6,-2.6,-7.5,0.0,102530.0,8.0,1979,01,02
2,5.0,0.0,13.0,1.3,-2.8,-7.2,0.0,102050.0,4.0,1979,01,03
3,8.0,0.0,13.0,-0.3,-2.6,-6.5,0.0,100840.0,2.0,1979,01,04
4,6.0,2.0,29.0,5.6,-0.8,-1.4,0.0,102250.0,1.0,1979,01,05
...,...,...,...,...,...,...,...,...,...,...,...,...
15336,1.0,0.9,32.0,7.5,7.5,7.6,2.0,98000.0,0.0,2020,12,27
15337,7.0,3.7,38.0,3.6,1.1,-1.3,0.2,97370.0,0.0,2020,12,28
15338,7.0,0.0,21.0,4.1,2.6,1.1,0.0,98830.0,0.0,2020,12,29
15339,6.0,0.4,22.0,5.6,2.7,-0.1,0.0,100200.0,0.0,2020,12,30
